# MNIST Dataset Processing and Neural Network

In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler


2024-12-07 10:38:33.185467: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-07 10:38:33.189243: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-07 10:38:33.200164: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733567913.220071    1063 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733567913.225277    1063 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-07 10:38:33.247494: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

## Step 1: Download and Process MNIST Dataset

In [2]:
# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Resize images to 16x16 (256 pixels)
x_train = x_train[:, ::2, ::2]  # Downsample by skipping every other pixel
x_test = x_test[:, ::2, ::2]

# Normalize pixel values to range [0.0, 1.0] (0.0 for white, 1.0 for black)
x_train = 1 - x_train / 255.0
x_test = 1 - x_test / 255.0

# Flatten images into 256-pixel vectors
x_train_flat = x_train.reshape(x_train.shape[0], -1)
x_test_flat = x_test.reshape(x_test.shape[0], -1)

# Convert targets to one-hot encoding
y_train_onehot = to_categorical(y_train, 10)
y_test_onehot = to_categorical(y_test, 10)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


## Step 2: Save Processed Data to CSV Files

In [3]:
# Save input data to input.csv
input_df = pd.DataFrame(x_train_flat)
input_df.to_csv("input.csv", index=False)

# Save target data to target.csv
target_df = pd.DataFrame(y_train_onehot)
target_df.to_csv("target.csv", index=False)

print("Saved input.csv and target.csv.")

Saved input.csv and target.csv.


## Step 3: Create a Neural Network using Keras

In [19]:
import subprocess

# Define parameters for `nn_generator`
model_directory = "./trained_model"
# create the trained model directory
import os
os.makedirs(model_directory, exist_ok=True)

input_file = "input.csv"
target_file = "target.csv"
training_verification_ratio = 0.7
learning_rate = 0.01
epochs = 100
tolerance = 0.1
num_generations = 100
log_level = 1
population_size = 4
num_offsprings = 10

# Create the command for training the neural network
command = [
    "./nn_generator",
    "--model-directory", model_directory,
    "--input-file", input_file,
    "--target-file", target_file,
    "--training-verification-ratio", str(training_verification_ratio),
    "--learning-rate", str(learning_rate),
    "--epochs", str(epochs),
    "--tolerance", str(tolerance),
    "--num-generations", str(num_generations),
    "--log-level", str(log_level),
    "--population-size", str(population_size),
    "--num-offsprings", str(num_offsprings)
]

# Run the command
try:
    #subprocess.run(command, check=True)
    # Create a copy of the current environment variables and add RUST_BACKTRACE=1
    env = os.environ.copy()
    env["RUST_BACKTRACE"] = "1"

    # Run the command
    subprocess.run(command, check=True, env=env)
    print("Model training completed successfully and saved in:", model_directory)
except subprocess.CalledProcessError as e:
    print("Error occurred during model training:", e)


Inputs: 59999 x 196
Targets: 59999 x 10
Training neural network with shape: NeuralNetworkShape { layers: [LayerShape { layer_type: Dense { input_size: 196, output_size: 10 }, activation: Sigmoid }] }
Epoch: 100
Inputs: 59999 x 196
Targets: 59999 x 10
Training neural network with shape: NeuralNetworkShape { layers: [LayerShape { layer_type: Dense { input_size: 196, output_size: 10 }, activation: Sigmoid }] }
Epoch: 100
Inputs: 59999 x 196
Targets: 59999 x 10
Training neural network with shape: NeuralNetworkShape { layers: [LayerShape { layer_type: Dense { input_size: 196, output_size: 10 }, activation: Tanh }] }
Epoch: 100
Inputs: 59999 x 196
Targets: 59999 x 10
Training neural network with shape: NeuralNetworkShape { layers: [LayerShape { layer_type: Dense { input_size: 196, output_size: 10 }, activation: Sigmoid }] }
Epoch: 100
Loss: 0.37950671970632377
Epoch: 100
Loss: 0.37950671970632377
Epoch: 100
Loss: 1.6913241645323618
Epoch: 100
Loss: 0.37950671970632377
Epoch: 100
Loss: 0.2561

thread '<unnamed>' panicked at src/gen/challenge/nn_challenge.rs:29:10:
called `Result::unwrap()` on an `Err` value: "Invalid neural network shape: NeuralNetworkShape { layers: [LayerShape { layer_type: Dense { input_size: 196, output_size: 10 }, activation: Sigmoid }, LayerShape { layer_type: Dense { input_size: 196, output_size: 168 }, activation: Sigmoid }, LayerShape { layer_type: Dense { input_size: 168, output_size: 196 }, activation: ReLU }] }"
stack backtrace:
   0: rust_begin_unwind
   1: core::panicking::panic_fmt
   2: core::result::unwrap_failed
   3: <learn::gen::challenge::nn_challenge::NeuralNetworkChallenge as learn::evol::evolution::challenge::Challenge<learn::gen::pheno::nn_pheno::NeuralNetworkPhenotype>>::score
   4: core::ops::function::impls::<impl core::ops::function::FnMut<A> for &F>::call_mut
   5: rayon_core::join::join_context::{{closure}}
   6: rayon::iter::plumbing::bridge_producer_consumer::helper
   7: rayon_core::job::StackJob<L,F,R>::run_inline
   8: ray

Loss: 0.1873219468697596
Epoch: 100
Loss: 1
Epoch: 100
Loss: 1
Epoch: 100
Loss: 0.18726283962748028
Epoch: 100
Loss: 1
Epoch: 100
Loss: 1
Loss: 0.1872046500087385
Epoch: 100
Loss: 1
Inputs: 59999 x 196
Targets: 59999 x 10
Training neural network with shape: NeuralNetworkShape { layers: [LayerShape { layer_type: Dense { input_size: 196, output_size: 10 }, activation: Tanh }] }
Epoch: 100
Loss: 0.1871473559069822
Epoch: 100
Loss: 1.6913241645323618
Epoch: 100
Loss: 0.1870909359788384
Epoch: 100
Loss: 1.659660677832816
Epoch: 100
Loss: 0.18703536960934136
Epoch: 100
Loss: 1.6513927514272417
Epoch: 100
Loss: 0.18698063687914698
Loss: 1.6456738405104643
Epoch: 100
Loss: 1.6414422867767064
Epoch: 100
Loss: 1.6404925358575446
Epoch: 100
Loss: 1.639083367230673
Epoch: 100
Loss: 1.6402130790702258
Epoch: 100
Loss: 1.6363079813218153
Epoch: 100
Loss: 1.6373136080337027
Epoch: 100
Loss: 1.6388917812900259
Epoch: 100
Loss: 1.6328715741630146
Epoch: 100
Loss: 1.6352756970090339
Epoch: 100
Loss: 1.6

## Step 4: Evaluate the Model

In [ ]:
# Evaluate on test data
loss, accuracy = model.evaluate(x_test_flat, y_test_onehot)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


NameError: name 'model' is not defined